In [6]:
import pandas as pd
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [7]:
os.chdir("E:\Insofe\PH.D\PHD_ML")

In [8]:
import turicreate

ModuleNotFoundError: No module named 'turicreate'

In [3]:
# pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# You can check the column names from the readme file

#Reading ratings file:
r_cols = ['userid', 'Hotelid', 'OverallRating']
ratings = pd.read_csv("user_hotel_rating-1555730075105.csv")

#Reading items file:
i_cols = ['Hotelid', 'userid' ,'reviewtext','Date', 'NoOfReaders', 'HelpfulToNoOfreaders', 'Value_rating', 'Rooms_rating',
'Location_rating', 'Cleanliness_rating', 'Checkin_rating', 'Service_rating', 'Businessservice_rating', 'AverageOverallRatingOfHotel', 'AveragePricing']
items = pd.read_csv("NewHotels_CutstomerVisitsdata-1554810098964.csv", parse_dates=[3])

In [4]:
# Ratings
print(ratings.shape)
ratings.head()

(271935, 3)


,userid,Hotelid,OverallRating
0,user_78131,hotel_558,3
1,user_78131,hotel_603,2
2,user_78131,hotel_610,3
3,user_78131,hotel_574,3
4,user_78131,hotel_570,3


In [5]:
ratings.dtypes

userid           object
Hotelid          object
OverallRating     int64
dtype: object

In [6]:
ratings.isnull().sum()

userid           0
Hotelid          0
OverallRating    0
dtype: int64

In [7]:
# Items
print(items.shape)
items.head()

(2469, 15)


,Hotelid,userid,reviewtext,Date,NoOfReaders,HelpfulToNoOfreaders,Value_rating,Rooms_rating,Location_rating,Cleanliness_rating,Checkin_rating,Service_rating,Businessservice_rating,AverageOverallRatingOfHotel,AveragePricing
0,hotel_104,hotel_4342,Awesome Everything about this resort was great...,2009-01-05,0,0,6,6,6,6,6,6,6,4.0,$227
1,hotel_104,hotel_2024,Convenient When I say Above Average I'm compa...,2008-11-10,0,0,4,4,6,5,4,5,1,4.0,$227
2,hotel_104,hotel_8720,Nice with its quirks Beach looking west,2008-11-03,0,0,3,4,4,5,6,4,4,4.0,$227
3,hotel_104,hotel_3267,Always a 5 star stay! Its always a good trip w...,2008-09-30,0,0,6,6,6,6,6,6,6,4.0,$227
4,hotel_104,hotel_5419,Loved It! The hotel is beautiful with spectacu...,2008-08-29,3,2,6,4,6,6,6,6,6,4.0,$227


In [8]:
#Building collaborative filtering model from scratch
n_users = ratings.userid.unique().shape[0]
print(n_users)
n_items = ratings.Hotelid.unique().shape[0]
print(n_items)

5010
130


In [9]:
from scipy.sparse import csr_matrix
data_matrix = np.zeros((n_users, n_items))
data_matrix_pivot = ratings.pivot(index = 'userid', columns ='Hotelid', values = 'OverallRating').fillna(0)
data_matrix = data_matrix_pivot.as_matrix()

C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [10]:
data_matrix

array([[0., 3., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 3., 3.],
       [0., 0., 0., ..., 0., 2., 0.],
       ...,
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 3., 2., ..., 2., 0., 2.],
       [0., 3., 0., ..., 4., 0., 0.]])

In [11]:
data_matrix.shape

(5010, 130)

In [12]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(data_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [13]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [14]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        print(ratings.mean)
        print(mean_user_rating)
        print(ratings - mean_user_rating[:, np.newaxis])
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [15]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

<built-in method mean of numpy.ndarray object at 0x00000220414C8CB0>
[1.13846154 1.33076923 1.13076923 ... 1.33846154 1.34615385 1.3       ]
[[-1.13846154  1.86153846 -1.13846154 ...  1.86153846 -1.13846154
  -1.13846154]
 [-1.33076923 -1.33076923 -1.33076923 ... -1.33076923  1.66923077
   1.66923077]
 [-1.13076923 -1.13076923 -1.13076923 ... -1.13076923  0.86923077
  -1.13076923]
 ...
 [-1.33846154 -1.33846154 -1.33846154 ... -1.33846154  1.66153846
  -1.33846154]
 [-1.34615385  1.65384615  0.65384615 ...  0.65384615 -1.34615385
   0.65384615]
 [-1.3         1.7        -1.3        ...  2.7        -1.3
  -1.3       ]]


In [16]:
!pip install turicreate

  Using cached https://files.pythonhosted.org/packages/db/54/167837569bcb816b3fe68f003f18d07ab9d5ac31b2b12b8f9b07b1ccc7a4/turicreate-4.2.tar.gz
  Complete output from command C:\Users\prash\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\prash\\AppData\\Local\\Temp\\pip-install-rwy8vdn5\\turicreate\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\prash\AppData\Local\Temp\pip-wheel-vhx27pe4 --python-tag cp36:
  running bdist_wheel
  running build
  installing to build\bdist.win-amd64\wheel
  running install
  
  
          ERROR
  
          If you see this message, pip install did not find an available binary package
          for your system. Supported platforms are:
  
          * Linux x86_64 (including WSL on Windows 10).
          * macOS 10.12+ x86_64.
          * Python 2.7, 3.5, or 3.6.
  
          Other possible causes of this error a

  Failed building wheel for turicreate
Command "C:\Users\prash\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\prash\\AppData\\Local\\Temp\\pip-install-rwy8vdn5\\turicreate\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\prash\AppData\Local\Temp\pip-record-4vsjbwrl\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\prash\AppData\Local\Temp\pip-install-rwy8vdn5\turicreate\


In [2]:
import turicreate

ModuleNotFoundError: No module named 'turicreate'

In [102]:
 popularity_model = turicreate.popularity_recommender.create(ratings, user_id='user_id', item_id='movie_id', target='rating')

NameError: name 'turicreate' is not defined